# Arquitetura

Uma CPPN treinada com uma GAN e associada a um NSGA-II:
- CPPN: Gera padrões, controlado por parâmetros (pesos, ativações, topologia);
- NSGA-II: Algoritmo evolutivo multiobjetivo, evolui os parâmetros da CPPN;
- GAN: Adversarial training, fornece feedback “perceptual” à CPPN, o discriminador julga se os padrões são realistas;

# Ideia de Aplicação

Este script gera "arte" usando um algoritmo genético (NSGA-II) para "evoluir" redes neurais especiais chamadas CPPNs. Primeiro, ele baixa o enorme dataset WikiArt para treinar um "crítico de arte" (um Discriminador). Uma vez que o crítico aprende a diferenciar arte real de falsa, o algoritmo NSGA-II evolui uma população de CPPNs por 20 gerações, recompensando aquelas que criam imagens (64x64) que o crítico considera "realistas" e que ao mesmo tempo são visualmente "diversificadas". Ao final, o script salva as 10 melhores imagens abstratas resultantes desse processo evolutivo na pasta out_evolved.



# Importações de Bibliotecas

In [2]:
!pip install torch torchvision numpy matplotlib pillow pymoo

In [4]:
!pip install pymoo #Biblioteca pymoo para utilizar o GA multiobjetivo

In [11]:
import math
import os
import copy
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination import get_termination

from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

# Importação do Dataset

Composto por 81.444 peças de arte divido em 129 artírtas, 11 gêneros e 27 classes de estilo;

Link: https://huggingface.co/datasets/huggan/wikiart?utm_source=chatgpt.com

In [12]:
# from datasets import load_dataset

# ds = load_dataset("huggan/wikiart")

# Funções e Classes

## Adaptação do dataset bruto:
Esta classe adapta o dataset bruto para o PyTorch, garantindo que cada imagem seja RGB e aplicando as transformações necessárias, como redimensionamento e normalização, antes de enviá-la para o treinamento.

In [ ]:
class WikiArtDataset(Dataset):
    """Wrapper para o dataset do Hugging Face para aplicar transformações."""
    def __init__(self, hf_dataset, transform):
        self.hf_dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        # Garante que a imagem é RGB
        img = self.hf_dataset[idx]['image'].convert('RGB')
        return self.transform(img)

## Treinar o Discriminador (GAN):

Esta função treina o "crítico de arte" (Discriminador) baixando o dataset e ensinando-o a diferenciar as imagens de arte "reais" das imagens "falsas" geradas aleatoriamente pelas CPPNs.

In [ ]:
def train_discriminator(discriminator, cppn_template, device, img_size=64, out_channels=3, batch_size=32, epochs=5):
    print("Iniciando pré-treinamento do Discriminador...")

    # 1. Configurar transformações
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalização padrão de GAN
    ])

    print("Carregando WikiArt")
    try:
        raw_dataset = load_dataset("huggan/wikiart", split='train')
    except Exception as e:
        print(f"Falha ao carregar dataset: {e}. Tente verificar sua conexão.")
        return discriminator

    train_dataset = WikiArtDataset(raw_dataset, transform)
    dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

    # 3. Configurar otimizador e perda
    optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    criterion = nn.BCELoss()

    genome_size = count_params(cppn_template)

    for epoch in range(epochs):
        for i, real_images in enumerate(dataloader):

            current_batch_size = real_images.size(0)
            real_images = real_images.to(device)

            # --- 1. Treinar com Imagens REAIS ---
            discriminator.zero_grad()
            labels = torch.full((current_batch_size, 1), 1.0, device=device)

            output = discriminator(real_images)
            loss_real = criterion(output, labels)
            loss_real.backward()

            # --- 2. Treinar com Imagens FALSAS ---
            # Gerar um lote de imagens falsas de CPPNs aleatórias
            fake_images_list = []
            for _ in range(current_batch_size):
                # Cria uma CPPN com genoma aleatório
                random_genome = np.random.uniform(-5.0, 5.0, size=genome_size).astype(np.float32)
                rand_cppn = copy.deepcopy(cppn_template)
                unpack_vector_to_model(rand_cppn, random_genome)
                rand_cppn.to(device)

                # Gera imagem
                img_np = cppn_generate_image(rand_cppn, size=img_size, out_channels=out_channels, device=device)

                # Converte imagem numpy para tensor
                arr = np.asarray(img_np).astype(np.float32) / 255.0
                arr = arr.transpose(2,0,1)[None, ...] # (1, C, H, W)
                fake_images_list.append(torch.from_numpy(arr))

            fake_images = torch.cat(fake_images_list, dim=0).to(device).float()

            # Aplicar normalização de GAN às imagens falsas
            fake_images = (fake_images - 0.5) / 0.5

            labels.fill_(0.0)
            output = discriminator(fake_images.detach())
            loss_fake = criterion(output, labels)
            loss_fake.backward()

            # --- 3. Atualizar Pesos ---
            loss_d = loss_real + loss_fake
            optimizer.step()

            if i % 100 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Batch [{i+1}/{len(dataloader)}], "
                      f"Loss D: {loss_d.item():.4f} (Real: {loss_real.item():.4f}, Fake: {loss_fake.item():.4f})")

    print("Treinamento do Discriminador concluído.")
    return discriminator

## Criação da CPPN:

Este código define a rede neural (CPPN) que, ao receber coordenadas de pixels (x, y), usa as funções matemáticas Seno e Gaussiana para gerar os padrões abstratos e coloridos (RGB) que formam a imagem final.

In [ ]:
# Funções de ativação
class Sin(nn.Module):
    def forward(self, x):
        return torch.sin(x)

class Gaussian(nn.Module):
    def forward(self, x):
        return torch.exp(-x**2)

class CPPN(nn.Module):
    def __init__(self, out_channels=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 8),
            Sin(),
            nn.Linear(8, 8),
            Gaussian(),
            nn.Linear(8, out_channels),
            nn.Sigmoid()
        )

    def forward(self, coords):
        return self.net(coords)

Essas funções traduzem os pesos da rede neural (CPPN) para um vetor 1D (genoma) que o algoritmo genético (NSGA-II) possa entender, e depois "traduzem" o genoma evoluído de volta para uma rede neural para que ela possa gerar uma imagem.

In [ ]:
def count_params(model):
    return sum(p.numel() for p in model.parameters())


def pack_params_to_vector(model):
    """Retorna um vetor 1D com todos os parâmetros do modelo"""
    with torch.no_grad():
        params = [p.view(-1).cpu().numpy() for p in model.parameters()]
    return np.concatenate(params).astype(np.float32)


def unpack_vector_to_model(model, vector):
    """Escreve os valores do vetor (numpy) nos parâmetros do modelo"""
    pointer = 0
    with torch.no_grad():
        for p in model.parameters():
            num = p.numel()
            slice_ = vector[pointer: pointer + num]
            p.copy_(torch.from_numpy(slice_.reshape(p.shape)).to(p.device))
            pointer += num
    assert pointer == len(vector)

## Configuração da GAN:

Este código define a arquitetura do Discriminador como uma rede neural convolucional simples que recebe uma imagem e devolve um único "score" entre 0 (totalmente falso) e 1 (totalmente real).

In [ ]:
class SimpleDisc(nn.Module):
    def __init__(self, in_channels=3):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 16, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(16, 32, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.AdaptiveAvgPool2d(1)
        )
        self.head = nn.Linear(64, 1)

    def forward(self, x):
        # x: (B, C, H, W)
        f = self.features(x).view(x.size(0), -1)
        return torch.sigmoid(self.head(f))  # [0,1] score

## Criação das Imagens:

A (make_coordinate_grid) cria uma lista com as coordenadas (x, y) de cada pixel, nesse caso, 4096 coordenadas para uma imagem 64x64.

A (cppn_generate_image) pega essa lista de coordenadas, passa ela pela rede CPPN, e recebe de volta a cor (RGB) para cada pixel. Por fim, ela junta essas cores para montar a imagem final e a converte para o formato 0-255.

In [ ]:
def make_coordinate_grid(size, device='cpu'):
    xs = np.linspace(-1, 1, size)
    ys = np.linspace(-1, 1, size)
    X, Y = np.meshgrid(xs, ys)
    coords = np.stack([X.ravel(), Y.ravel()], axis=-1).astype(np.float32)
    return torch.from_numpy(coords).to(device)


def cppn_generate_image(cppn_model, size=64, out_channels=1, device='cpu'):
    coords = make_coordinate_grid(size, device=device)
    with torch.no_grad():
        out = cppn_model(coords)
    img = out.cpu().numpy().reshape(size, size, out_channels)

    img = (img * 255).clip(0, 255).astype(np.uint8)
    if out_channels == 1:
        img = img.squeeze(-1)
    return img

## Objetivos da Evolução:

(discriminator_score_from_image) calcula o score de realismo, o quão artística a imagem parece para o crítico, e (diversity_metric) calcula a complexidade visual da imagem pelo desvio padrão dos pixels.

In [ ]:
def discriminator_score_from_image(discriminator, image_np, device='cpu'):
    arr = np.asarray(image_np).astype(np.float32) / 255.0
    if arr.ndim == 2:
        arr = arr[None, None, ...]  # (1,1,H,W)
    else:
        arr = arr.transpose(2,0,1)[None, ...]

    tensor = torch.from_numpy(arr).to(device).float()
    tensor = (tensor - 0.5) / 0.5

    with torch.no_grad():
        score = discriminator(tensor).cpu().item()
    return score


def diversity_metric(image_np):
    arr = np.asarray(image_np).astype(np.float32) / 255.0
    return float(arr.std())

## Teste de Sobrevivencia:

Esta classe define o teste de aptidão para o algoritmo genético. Para cada genoma da população, ela gera a imagem correspondente, calcula as notas de realismo e diversidade e as entrega ao otimizador, que então decide quais são os melhores.

In [ ]:
class CPPN_Evolution_Problem(Problem):
    def __init__(self, genome_size, discriminator, cppn_template, size=64, out_channels=1, device='cpu'):
        super().__init__(n_var=genome_size, n_obj=2, n_constr=0, xl=-5.0, xu=5.0)
        self.discriminator = discriminator
        self.cppn_template = cppn_template
        self.size = size
        self.out_channels = out_channels
        self.device = device

    def _evaluate(self, X, out, *args, **kwargs):

        pop_size = X.shape[0]
        f1 = np.zeros((pop_size,))
        f2 = np.zeros((pop_size,))

        for i in range(pop_size):
            genome = X[i]
            cppn = copy.deepcopy(self.cppn_template)

            unpack_vector_to_model(cppn, genome)
            cppn.to(self.device)
            img = cppn_generate_image(cppn, size=self.size, out_channels=self.out_channels, device=self.device)

            try:
                score = discriminator_score_from_image(self.discriminator, img, device=self.device)
            except Exception:
                score = 0.0

            div = diversity_metric(img)
            f1[i] = -score
            f2[i] = -div

        out['F'] = np.column_stack([f1, f2])

## Principal:

Cria os modelos CPPN e Discriminador e configura o problema de evolução (teste de aptidão) e inicia o algoritmo NSGA-II para evoluir os artistas CPPN, devolvendo os melhores resultados no final.

In [ ]:
def run_evolution(pop_size=20, generations=30, img_size=64, out_channels=3, device='cpu'):

    DISCRIMINATOR_PATH = "discriminator_wikiart.pth"

    cppn_template = CPPN(out_channels=out_channels).to(device)
    genome_size = count_params(cppn_template)
    print(f"Genome size: {genome_size} (para {out_channels} canais)")

    discriminator = SimpleDisc(in_channels=out_channels).to(device)

    if os.path.exists(DISCRIMINATOR_PATH):
        print(f"Carregando discriminador pré-treinado de {DISCRIMINATOR_PATH}...")
        discriminator.load_state_dict(torch.load(DISCRIMINATOR_PATH))
    else:
        print("Nenhum discriminador pré-treinado encontrado. Iniciando treinamento...")

        discriminator = train_discriminator(discriminator,
                                          cppn_template,
                                          device,
                                          img_size=img_size,
                                          out_channels=out_channels,
                                          epochs=5)


        torch.save(discriminator.state_dict(), DISCRIMINATOR_PATH)
        print(f"Discriminador treinado e salvo em {DISCRIMINATOR_PATH}")

    # Coloca o discriminador em modo de avaliação
    discriminator.eval()

    # 4) definir problema
    problem = CPPN_Evolution_Problem(genome_size=genome_size,
                                     discriminator=discriminator,
                                     cppn_template=cppn_template,
                                     size=img_size,
                                     out_channels=out_channels,
                                     device=device)

    # 5) algoritmo NSGA-II
    algorithm = NSGA2(pop_size=pop_size)
    termination = get_termination("n_gen", generations)

    print("\nIniciando evolução (NSGA-II)...")
    res = minimize(problem,
                   algorithm,
                   termination,
                   seed=1,
                   save_history=False,
                   verbose=True)

    return res

## Iniciar:

Define os parâmetros principais, como tamanho da população e número de gerações, chama a função (run_evolution) para executar todo o processo de evolução e pega os 10 melhores genomas resultantes e os utiliza para gerar e salvar as imagens finais na pasta out_evolved.

In [ ]:
if __name__ == '__main__':
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print('Device:', device)

    POP = 30
    GENS = 20
    IMG_SIZE = 64
    OUT_CH = 3

    res = run_evolution(pop_size=POP, generations=GENS, img_size=IMG_SIZE, out_channels=OUT_CH, device=device)

    os.makedirs('out_evolved', exist_ok=True)
    for i, genome in enumerate(res.X[:10]):
        cppn_template = CPPN(out_channels=OUT_CH)
        unpack_vector_to_model(cppn_template, genome)
        img = cppn_generate_image(cppn_template, size=IMG_SIZE, out_channels=OUT_CH, device=device)
        im = Image.fromarray(img)
        im.save(f'out_evolved/ind_{i}.png')

    print('Evolução finalizada. Imagens salvas em ./out_evolved/')